### 1
1) Для каждого товара вывести следующую информацию
 - код товара
 - название товара
 - бренд
 - категорию
 - сумму продаж в шт, за последний полный месяц
 - сумму продаж в рублях, за последний полный месяц
 - среднюю цену на товара, за последний полный месяц
 - средний объем продаж в штуках за последние 3 месяца (включая последний месяц)
 - разницу между текущим остатком и средним объемом продаж в штуках за последние 3 месяца

В итоговом результате должны быть только товары, у которых были продажи за последний месяц.

Прошлый месяц должен прописываться в коде, нахождением этого месяца, а не "2025-05-01"


In [ ]:
WITH
    toStartOfMonth(today()) AS m0,
    addMonths(m0, -1) AS m1,
    addMonths(m0, -3) AS m3
SELECT
    p.itemid,
    p.name_ax,
    p.brand_name_ax,
    p.category,
    lm.qty_m1,
    lm.rub_m1,
    lm.avg_price_m1,
    ROUND((m3s.qty_3m / 3.0), 2) AS avg_qty_3m,
    ROUND(st.qty_stock - (m3s.qty_3m / 3.0), 2) AS stock_minus_avg_qty_3m
FROM dwh.products p
INNER JOIN
(
    SELECT
        itemid,
        sum(qty) AS qty_m1,
        sum(qty * price) AS rub_m1,
        sum(qty * price) / nullIf(sum(qty), 0) AS avg_price_m1
    FROM dwh.sales
    WHERE date >= m1 AND date < m0
    GROUP BY itemid
    HAVING qty_m1 > 0
) lm ON lm.itemid = p.itemid
LEFT JOIN
(
    SELECT
        itemid,
        sum(qty) AS qty_3m
    FROM dwh.sales
    WHERE date >= m3 AND date < m0
    GROUP BY itemid
) m3s ON m3s.itemid = p.itemid
LEFT JOIN
(
    SELECT
        itemid,
        argMax(qty, date) AS qty_stock
    FROM dwh.stock
    GROUP BY itemid
) st ON st.itemid = p.itemid;


### 2
2) Вывести топ 5 брендов  по объему продаж в рублях за последний месяц, сравнить с объемом продаж за предыдущий месяц.

In [ ]:
WITH
    toStartOfMonth(today()) AS m0,
    addMonths(m0, -1) AS m1,
    addMonths(m0, -2) AS m2
SELECT
    brand,
    rub_m1,
    rub_m2,
    (rub_m1 - ifNull(rub_m2, 0)) AS diff_rub,
    (rub_m1 / nullIf(ifNull(rub_m2, 0), 0) - 1) AS pct_change
FROM
(
    SELECT
        p.brand_name_ax AS brand,
        sumIf(s.qty * s.price, s.date >= m1 AND s.date < m0) AS rub_m1,
        sumIf(s.qty * s.price, s.date >= m2 AND s.date < m1) AS rub_m2
    FROM dwh.sales s
    INNER JOIN dwh.products p ON p.itemid = s.itemid
    GROUP BY brand
)
ORDER BY rub_m1 DESC
LIMIT 5;